## Chaine de traitement en Python pour adapter le moteur de reconnaissance d'entités nommées de Spacy par entrainement. 

Il s'agit de :


1.   annoter automatiquement en noms de lieux (LOC) et de personnes (PERS) un extrait du texte "Bel Ami" de Maupassant en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy ;
2.   transférer le texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations créées automatiquement;
3.   Corriger manuellement ces annotations en vue de créer un jeu  de données d'entrainement ;
4.   Formatter et intégrer les annotations corrigées dans Spacy en tant que jeu de données d'entrainement ;
5.  Initialiser un modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement ;
6.  Initialiser l'entrainement avec les paramètres définis dans le fichier .cfg et entrainement par le biais du CLI de Spacy ;
7.  Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant.





**Prérequis : Installation ou mise à jour de Spacy et téléchargement du modèle**

In [ ]:
!python -m spacy info

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

In [ ]:
!python -m spacy download fr_core_news_sm  

**Chargement d'un extrait du premier chapiture de Bel Ami de Maupassant.**
A télécharger et enregistrer sur votre ordinateur à partir de : 
https://github.com/cvbrandoe/coursTAL/blob/master/2022/belAmi_01-01.txt 

In [ ]:
#from google.colab import files
#uploaded = files.upload()

**1. Annotation automatiquement en noms de lieux (LOC) et de personnes (PERS) du texte en entrée en s'appuyant sur le modèle pour le français "fr_core_news_sm" proposé par Spacy**

In [ ]:
import spacy
import json
import requests
import os

def get_class_id(label):
  """
  Translates the spaCy label id into the tagtog entity type id
  - label: spaCy label id
  """
  choices = {'PER': 'e_1', 'LOC': 'e_2', 'GPE': 'e_2'}
  return choices.get(label, None)

def get_entities(spans, pipeline):
  """
  Translates a tuple of named entity Span objects (https://spacy.io/api/span) into a 
  list of tagtog entities (https://docs.tagtog.net/anndoc.html#ann-json). Each entity is
  defined by the entity type ID (classId), the part name where the annotation is (part),
  the entity offsets and the confidence (annotation status, who created it and probabilty).
  - spans: the named entities in the spaCy doc
  - pipeline: trained pipeline name
  """
  default_prob = 1
  default_part_id = 's1v1'
  default_state = 'pre-added'
  tagtog_entities = []
  for span in spans:
    class_id = get_class_id(span.label_)
    if class_id is not None:
      tagtog_entities.append( {
        'classId': class_id,
        'part': default_part_id,
        'offsets':[{'start': span.start_char, 'text': span.text}],
        'confidence': {'state': default_state,'who': ['ml:' + pipeline],'prob': default_prob},
        'fields':{},
        # this is related to the kb_id (knowledge base ID) field from the Span spaCy object
        'normalizations': {}} )
  return tagtog_entities


In [ ]:

#text = "Quand Georges Duroy parvint au boulevard, il s’arrêta encore, indécis sur ce qu’il allait faire. Il avait envie maintenant de gagner les Champs-Élysées et l’avenue du bois de Boulogne pour trouver un peu d’air frais sous les arbres."
myfile = open("belAmi_01-01.txt").read().replace("’","'")

def read_text(file_name):
  with open (file_name, "r", encoding="utf_8") as myfile:
    lines = list(line for line in (l.strip() for l in myfile) if line)
  return str(lines)
#myfile = read_text("belAmi_01-01.txt").replace("’","'")

pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(myfile)

def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")             
show_ents(doc)


**2. transfer du texte annoté vers l'outil d'annotation TagTog via leur API REST afin de corriger manuellement les annotations**

Au préalable, il faut créer un compte utilisateur et un projet dans TagTog (https://www.tagtog.net/) et modifier les informations de connexion ci-dessous.

Lors de la configuration (settings) de votre projet sur TagTog, dans la rubrique entities, il faut déclarer les trois catégories d'entités pour l'exercice (dans cet ordre là) :  **Personnages**, **Lieux** et **Misc** (voir diapo #48 du support du cours : https://github.com/cvbrandoe/coursTAL/blob/master/2022/Cours%20TAL%20HN%20-%20ENC%20-%2023_02_2022%20.pdf).


In [ ]:
# Set the credentials at tagtog and project name
MY_USERNAME = 'ACHANGER'
MY_PASSWORD = 'ACHANGER'
MY_PROJECT = 'ACHANGER'

# API authentication
tagtogAPIUrl = "https://www.tagtog.net/-api/documents/v1"
auth = requests.auth.HTTPBasicAuth(username=MY_USERNAME, password=MY_PASSWORD)

# Initialize ann.json (specification: https://docs.tagtog.net/anndoc.html#ann-json)
annjson = {}
# Set the document as not confirmed, an annotator will manually confirm whether the annotations are correct
annjson['anncomplete'] = False
annjson['metas'] = {}
annjson['relations'] = []                      
# Transform the spaCy entities into tagtog entities
annjson['entities'] = get_entities(doc.ents, pipeline)

print(myfile)
print(json.dumps(annjson))

# Parameters for the API call 
# see https://docs.tagtog.net/API_documents_v1.html#examples-import-pre-annotated-plain-text-file
params = {'owner': MY_USERNAME, 'project': MY_PROJECT, 'output': 'null', 'format': 'default-plus-annjson'}
# Pre-annotated document composed of the content and the annotations
files=[('BelAmi_extrait0101.txt', myfile), ('BelAmi_extrait0101.ann.json', json.dumps(annjson))]
# POST request to send the pre-annotated document
response = requests.post(tagtogAPIUrl, params=params, auth=auth, files=files)

print(response.text)

**3.  Correction manuelle des annotations en vue de créer un jeu  de données d'entrainement**

Ensuite télécharger fichier json à partir de tagtog et renommez-le en "BelAmi_01_01_corrected.ann.json", ce fichier est traité ci-dessous.

In [ ]:
from google.colab import files
uploaded = files.upload()


**4. Formattage et intégration des annotations corrigées dans Spacy en tant que jeu de données d'entrainement**

In [ ]:
# importing the module
import json
  # reading the data from the file
with open('BelAmi_01_01_corrected.ann.json') as f:
    data = f.read()  
print("Data type before reconstruction : ", type(data))
# reconstructing the data as a dictionary
js_corrected_training_data = json.loads(data)
print("Data type after reconstruction : ", type(js_corrected_training_data))
print(js_corrected_training_data)

In [ ]:
def get_class_id_inv(label):
  """
  Translates the tagtog entity type id into spaCy label id 
  - label: tagtog entity type id
  """
  choices = {'e_1': 'PER', 'e_2': 'LOC', 'e_3': 'MISC'}
  return choices.get(label, None)

In [ ]:


training_data = {'classes' : ['PER', "LOC", "MISC"], 'annotations' : []}
temp_dict = {}
temp_dict['text'] = myfile
temp_dict['entities'] = []
for example in js_corrected_training_data['entities']:  
  #print(example)
  start = example['offsets'][0]['start']
  end = int(example['offsets'][0]['start']) + len(example['offsets'][0]['text'])
  label = get_class_id_inv(example['classId'])
  temp_dict['entities'].append((start, end, label))
  #print(temp_dict['entities'])
training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

**5.  Initialisation du modèle qui sera entrainé de manière incrémentale à partir du jeu d'entrainement** 

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("fr") # load a new spacy model
doc_bin = DocBin() # create a DocBin object


In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object


Le fichier de configuration base_config_nermodel.cfg est à télécharger à partir d'ici : https://github.com/cvbrandoe/coursTAL/blob/master/2022/base_config_nermodel.cfg 

In [ ]:
from google.colab import files
uploaded = files.upload()

**6.  Initialisation de l'entrainement avec les paramètres définis (hyperparamètres) dans le fichier .cfg et entrainement par le biais du CLI de Spacy**

In [ ]:
!python -m spacy init fill-config base_config_nermodel.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --output ./


Télécharger l'autre extrait de Bel Ami pour tester le modèle à partir de : https://github.com/cvbrandoe/coursTAL/blob/master/2022/belAmi_01-05.txt

In [ ]:
from google.colab import files
uploaded = files.upload()

**7. Test du nouveau modèle sur un autre extrait de "Bel Ami" de Maupassant**

In [ ]:
nlp_ner = spacy.load("model-best")

test_dataset = open("belAmi_01-05.txt").read().replace("’","'")
doc = nlp_ner(test_dataset)

colors = {"PER": "#F67DE3", "LOC": "#7DF6D9", "MISC" : "#FFFFFF"}
options = {"colors": colors} 

show_ents(doc)

#spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

**A titre de comparaison, test avec ce même texte mais à partir du modèle de base fourni par Spacy, fr_core_news_sm.**

In [ ]:
pipeline = 'fr_core_news_sm' 
nlp = spacy.load(pipeline)
doc = nlp(test_dataset)
show_ents(doc)

Remarque 1. lors de la mise à jour du modèle NER (idem pour d'autres couches comme POS) avec les annotations de l'utilisateur, un processus itératif se met en place afin de faire converger les prédictions faites (des poids) vers des valeurs optimales vis-à-vis des annotations de référence. A caque itération, on calcule le "gradient" qui correspond, en optimisation, aux dérivées partielles de la fonction de classification non linéaire recherchée (voir [1], vous trouverez aussi la notion de backpropagation clé dans ces approches de DL pour le TAL). Pour observer la manière dont la valeur du gradient évolue (et se stabilise) lors des itérations, il est conseillé de regarder le paramètre "losses" mis à jour par la méthode update(..., losses=losses).
[1]: https://web.stanford.edu/class/archive/cs/cs224n/cs224n.1162/handouts/CS224N_DeepNLP_Week7_lecture3.pdf 
Aussi : https://spacy.io/usage/training

Remarque 2. Pour éviter le surapprentissage (overfitting) d'un modèle : vous pouvez identifier que votre modèle n'est pas bon lorsqu'il fonctionne bien sur les données d'entraînement mais ne donne pas de bons résultats sur des données nouvelles et non encore vues. Autrement dit, le modèle "mémorise" les données d'apprentissage et n'est pas performant avec les nouvelles données.